## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/Weather_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Tubuala,PA,9.52,-79.03,86.00,74,40,6.93,scattered clouds
1,1,Hukuntsi,BW,-24.00,21.75,61.68,26,7,10.45,clear sky
2,2,Tuatapere,NZ,-46.13,167.68,55.00,64,100,8.01,light rain
3,3,Virginia Beach,US,36.85,-75.98,70.00,52,0,1.99,clear sky
4,4,Avarua,CK,-21.21,-159.78,75.20,83,100,10.29,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 69
What is the maximum temperature you would like for your trip? 99


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Tubuala,PA,9.52,-79.03,86.00,74,40,6.93,scattered clouds
3,3,Virginia Beach,US,36.85,-75.98,70.00,52,0,1.99,clear sky
4,4,Avarua,CK,-21.21,-159.78,75.20,83,100,10.29,overcast clouds
10,10,Saint George,US,37.10,-113.58,81.00,9,1,1.25,clear sky
12,12,Puerto Ayora,EC,-0.74,-90.35,77.00,57,20,21.92,few clouds
13,13,Rikitea,PF,-23.12,-134.97,72.75,79,0,17.90,clear sky
14,14,Kamina,CD,-8.74,24.99,73.13,55,0,3.96,clear sky
15,15,Zlobin,HR,45.29,14.65,69.01,77,75,2.24,broken clouds
21,21,Kaoma,ZM,-14.78,24.80,73.40,22,6,3.67,clear sky
23,23,Mazagao,BR,-0.12,-51.29,95.00,53,40,8.05,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Tubuala,PA,9.52,-79.03,86.0,74,40,6.93,scattered clouds
3,3,Virginia Beach,US,36.85,-75.98,70.0,52,0,1.99,clear sky
4,4,Avarua,CK,-21.21,-159.78,75.2,83,100,10.29,overcast clouds
10,10,Saint George,US,37.10,-113.58,81.0,9,1,1.25,clear sky
12,12,Puerto Ayora,EC,-0.74,-90.35,77.0,57,20,21.92,few clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Tubuala,PA,86.00,scattered clouds,9.52,-79.03,
3,Virginia Beach,US,70.00,clear sky,36.85,-75.98,
4,Avarua,CK,75.20,overcast clouds,-21.21,-159.78,
10,Saint George,US,81.00,clear sky,37.10,-113.58,
12,Puerto Ayora,EC,77.00,few clouds,-0.74,-90.35,
13,Rikitea,PF,72.75,clear sky,-23.12,-134.97,
14,Kamina,CD,73.13,clear sky,-8.74,24.99,
15,Zlobin,HR,69.01,broken clouds,45.29,14.65,
21,Kaoma,ZM,73.40,clear sky,-14.78,24.80,
23,Mazagao,BR,95.00,scattered clouds,-0.12,-51.29,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Virginia Beach,US,70.00,clear sky,36.85,-75.98,Hilton Virginia Beach Oceanfront
4,Avarua,CK,75.20,overcast clouds,-21.21,-159.78,Paradise Inn
10,Saint George,US,81.00,clear sky,37.10,-113.58,Best Western Plus Abbey Inn
12,Puerto Ayora,EC,77.00,few clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
13,Rikitea,PF,72.75,clear sky,-23.12,-134.97,Pension Maro'i


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))